# HOMEWORK 3

## 1) DATA COLLECTION

### 1.1 Get the list of animes
We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

The output of this step is a .txt file whose single line corresponds to an anime's url.

In [2]:
#import libraries
import requests
url = 'https://myanimelist.net/topanime.php'
response= requests.get(url)
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
#soup.find_all('a') #Finds all the links
#soup.find_all('tr') # Finds all the tables

In [4]:
def get_links_from_soup(soup):
    anime = []
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')))
    return anime

In [ ]:
tot_list= []
for lim in range(0, 20000, 50):
    if lim==0:
        new_url = url
    else:
        new_url = url+'?limit='+str(lim)
    response = requests.get(new_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tot_list += get_links_from_soup(soup)

In [ ]:
len(tot_list)

19122

In [ ]:
with open('out.txt', 'w') as f:
    for n, link in tot_list:
        f.write(link+'\n')

### 1.2. Crawl animes
Once you get all the urls in the first 400 pages of the list, you:

* Download the html corresponding to each of the collected urls.
* After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the * * * stopping point. More details in Important (2).
* Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

In [ ]:
with open("out.txt") as file:
    #the line will be the url
    number=0
    
    for url in file:
        respone=requests.get(url)
        soup=BeautifulSoup(response.content,"html.parser")
        soup=str(soup)
        #save file as htlm
        name=('article_%d.html' % (number))
        html_file = open(name,"w")
        html_file.write(soup)
        html_file.close()
        number+=1
       

**Important**

Due to the large amount of pages you need to download, follow the next tips that will help you speeding up several time-consuming operations.

[Save time downloading files] You are asked to crawl a considerable number of pages, which will take plenty of time. To speed up the operation, we suggest you to work in parallel with your group's colleagues or even generate code that works in parallel with all the CPUs available in your computer. In particular, using the same code, each component of the group can be in charge of downloading a subset of pages (e.g., the first 100). PAY ATTENTION: Once obtained all the pages, merge your results into an unique dataset. In fact, the search engine must look up for results in the whole set of documents.

[Save your data] It is not nice to restart a crawling procedure, given its runtime. For this reason, it is extremely important that for every time you crawl a page, you must save it with the name article_i.html, where i corresponds to the number of articles you have already downloaded. In such way, if something goes bad, you can restart your crawling procedure from the i+1-th document.

### 1.3 Parse downloaded pages
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

* **Anime Name** (to save as animeTitle): *String*
* **Anime Type** (to save as animeType): *String*
* **Number of episode** (to save as animeNumEpisode): *Integer*
* **Release and End Dates of anime** (to save as releaseDate and endDate): Convert both release and end date into *datetime format*.
* **Number of members** (to save as animeNumMembers): *Integer*
* **Score** (to save as animeScore): *Float*
* **Users** (to save as animeUsers): *Integer*
* **Rank** (to save as animeRank): *Integer*
* **Popularity** (to save as animePopularity): *Integer*
* **Synopsis** (to save as animeDescription): *String*
* **Related Anime** (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. *List of strings*.
* **Characters** (to save as animeCharacters): *List of strings*.
* **Voices** (to save as animeVoices): *List of strings*
* **Staff** (to save as animeStaff): Include the staff name and their responsibility/task in a *list of lists*.

For each anime, you create an anime_i.tsv file of this structure:

animeTitle \t animeType \t  ... \t animeStaff

## 2. SEARCH ENGINE
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

* Removing stopwords
* Removing punctuation
* Stemming
* Anything else you think it's needed

For this purpose, you can use the nltk library.

In [1]:
import pandas as pd
import string
import json
from operator import itemgetter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ninakaploukhaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ninakaploukhaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

In [3]:
df = pd.read_table("../shared_stuff/tsv_files/total_pages.tsv",
                       delimiter = "\t",
                       header = "infer",
                       on_bad_lines = "skip") # fix line 16473

In [4]:
def str_to_list(s):
    """
    This functions returns a list
    with each of the characters of the input string
    
    Arguments
        s : string
        
    Returns
        (list)
    """
    
    return [char for char in s]

def has_digits(s):
    """
    This function checks whether a string
    contains any digits
    
    Arguments
        s : string
        
    Returns
        (bool) True / False
    """
    
    return len([char for char in s if char.isdigit()]) != 0

def bad_words():
    """
    This function creates a list with words
    that should be excluded from the vocabulary
    during preprocessing, including punctuation,
    stopwords et similia
    
    Arguments
        none
        
    Returns
        (list)
    """
    
    punct = str_to_list(string.punctuation)
    punct += ["...", "''", "``", '""']
    
    stops = stopwords.words("english")
    
    other_suffixes = ["'s", "n't"]
    
    return punct + stops + other_suffixes

def preprocess(text, stemmer):
    """
    This function preprocesses some text (a document)
    by isolating each word, excluding stopwords et similia,
    and finally stemming them
    
    Arguments
        text : (string)
        stemmer : stemmer object, e.g. SnowBallStemmer()
    
    Returns
        (list) preprocessed input text
    """
    
    text = str(text).replace("/"," ") #between some words there is /, let's replace it with space
    
    tokens = word_tokenize(text)
        
    return [stemmer.stem(w) for w in tokens 
            if w not in bad_words() and not has_digits(w) and len(w) == len(w.encode())]

In [5]:
stemmer = SnowballStemmer("english")

df['synopsis_clean'] = df.apply(lambda row: preprocess(row['synopsis'], stemmer), 
                                axis = 'columns')

### 2.1.1) Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains the word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

In [6]:
def create_vocab(corpus):
    """
    This function creates a set of unique
    and preprocessed words from a corpus
    
    Arguments
        corpus : pandas df column or list-like
    
    Returns
        vocab  : dictionary with the words as keys
                 and a unique integer for each as values
    """
    
    vocab = set()
    
    for doc in corpus:
        vocab.update(set(doc))

    return {word:idx for idx, word in enumerate(vocab)}
    

def save_dict_to_file(dct, filename):
    """
    This function saves a dictionary 
    to an external JSON file
    
    Arguments
        dct       : dictionary
        filename  : name of the file
        
    Returns
        void
    """
        
    with open(filename, "w") as file:
        json.dump(dct, file)
        

def read_dict_from_file(filename):
    """
    This function reads a dictionary
    from an external JSON file
        
    Arguments
        filename : name of the file
    
    Returns
        dct : dictionary with the contents of 'filename'
    """

    with open(filename, "r") as file:
        dct = json.loads(file.read())

    return dct

In [7]:
# only execute this cell the first time or 
# when the preprocessing changes!

vocab = create_vocab(df['synopsis_clean'])

save_dict_to_file(vocab, "vocabulary.json")

In [8]:
vocab = read_dict_from_file("vocabulary.json")

In [9]:
def create_inv_idx(corpus, vocab): 
    """
    This functions creates an inverted index list
    given a corpus of documents and a vocabulary
    
    Arguments
        corpus  : pandas df column or list-like
        vocab   : dictionary of all the words in the corpus
    
    Returns
        inv_idx : dictionary with the words as referenced in 'vocab' as keys 
                  and the lists of the documents each word is in as values       
    """
    
    inv_idx = {}
    
    for idx, word in zip(vocab.values(), vocab.keys()):
        inv_idx[idx] = [doc_id for doc_id, doc in enumerate(corpus) if word in doc]
    
    return inv_idx

In [10]:
# only execute this cell the first time or 
# when the preprocessing/vocabulary change!

inv_idx = create_inv_idx(df['synopsis_clean'], vocab)

save_dict_to_file(inv_idx, "inv_idx.json")

In [11]:
# The keys of the dict we get are str, not int like when we created it
# I don't think it's necessary, but should we parse them when we read the json?
inv_idx = read_dict_from_file("inv_idx.json")

### 2.1.2) Execute the query
Given a query, that you let the user enter:

saiyan race
the Search Engine is supposed to return a list of documents.

What documents do we want?
Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

animeTitle
animeDescription
Url
Example Output:

animeTitle	animeDescription	Url
Fullmetal Alchemist: Brotherhood	...	https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
Gintama	...	https://myanimelist.net/anime/28977/Gintama%C2%B0
Shingeki no Kyojin Season 3 Part 2	...	https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2
If everything works well in this step, you can go to the next point, and make your Search Engine more complex and better in answering queries.

In [21]:
def parse_query(query, vocab, stemmer):
    """
    This functions converts the list of words
    input by the user into the list of the IDs
    the words are saved as in the vocabulary
    
    Arguments
        query   : list of words
        vocab   : vocabulary of words with the words as keys
                  and their IDs as values
        stemmer : stemmer object, e.g. SnowBallStemmer()
        
    Returns
        list of the IDs of the words in the query
    """
    
    parsed_query = []
    
    for word in query:
        try:
            parsed_query.append(vocab[stemmer.stem(word)])
        except KeyError:
            print(f"The term '{word}' wasn't found anywhere!")
    
    return parsed_query


def get_results(query, inv_dx):
    """
    This functions finds the documents all the words
    in the query are in.
    
    It finds them in three steps:
    1. creates a list of docs each word is in from the inverted index
    2. converts that list into a set
    3. intersects all those sets into a single set
       
    Arguments
        query   : list of words as parsed by 'parse_query'
        inv_idx : (dictionary) inverted index
        
    Returns
        set with the documents that contain all the words in the query
    """
    
    return set.intersection(*[set(inv_idx[str(q)]) for q in query])


def get_df_entries(df, results,
                   url_file = "../shared_stuff/url_list.txt", 
                   simil = None):
    """
    This function filters the dataset so it only shows
    the rows which match the results, and adds a new column
    with the URL for the anime of each row.
    
    Arguments
        df       : pandas dataframe
        results  : set with the row indices to be filtered out
        url_file : external file with the URLs for each of the rows in df
        simil    : dictionary with the similarity scores
    
    Returns
        df : filtered pandas dataframe
    """
    
    if not results:
        print("No results!")
        return pd.DataFrame()
    
    with open(url_file, 'r') as file:
        url_list = file.read().split("\n")

    df = df.iloc[[*results]]
    df = df[["title", "synopsis"]]
    df = df.rename(columns = {"title": "animeTitle", 
                              "synopsis": "animeDescription"})
    
    df['animeUrl'] = itemgetter(*results)(url_list)
    
    if simil is not None: #in case we need a colum for similiaty
        df["Similarity"] = itemgetter(*results)(simil)
        df = df.sort_values(by=['Similarity'], ascending = False)

    return df

In [13]:
query = parse_query(input().split(), vocab, stemmer)

# if at least one word in the query
# is in the vocabulary
if query:
    
    results = get_results(query, inv_idx)

    df_entries = get_df_entries(df, results)

    if df_entries.size != 0: 
        display(df_entries)

best anime


,animeTitle,animeDescription,animeUrl
15236,Choubakuretsu Ijigen Menko Battle: Gigant Shoo...,"The anime follows Tsukasa, whose dream is to b...",https://myanimelist.net/anime/42875/Chou_Futsu...
14725,Ad Lib Anime Kenkyuujo,"Emiri Katou and Kaori Fukuhara, the voice actr...",https://myanimelist.net/anime/48586/ABC_Tenkiy...
5510,Nurse Witch Komugi-chan Magikarte,"Ungrar, the King of Viruses, has escaped from ...",https://myanimelist.net/anime/1300/Omishi_Maho...
3846,Dr. Slump,"In Penguin Village, humans live alongside talk...",https://myanimelist.net/anime/5197/Dragon_League
16909,Kuakou de Qingwa,The story is about a proud frog who likes to b...,https://myanimelist.net/anime/48011/Kuaile_Xia...
10388,Tamala 2010: A Punk Cat in Space OVA,"After a long 5-year wait, the team that create...",https://myanimelist.net/anime/36162/Sou__Uchia...
13340,Shounen H ga Mita Sensou,An educational anime about the Pacific War. Ba...,https://myanimelist.net/anime/8829/Shounen_Ash...
8607,Heybot!,The anime takes place on the screw-shaped isla...,https://myanimelist.net/anime/3906/H_P_Lovecra...
14500,Yowai Robot to 10 no Story Project,"The project is inspired by ""weak robot"", it is...",https://myanimelist.net/anime/21129/Youtai_Nuh...
8359,Attraction,Interactive anti-smoking anime. Directed by Ko...,https://myanimelist.net/anime/2537/Angelique__...


### 2.2) Conjunctive query & Ranking score
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

* Find all the documents that contains all the words in the query.
* Sort them by their similarity with the query.
* Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use é * Python libraries) for maintaining the top-k documents.

To solve this task, you will have to use the tfIdf score, and the Cosine similarity. The field to consider it is still the synopsis. Let's see how.

### 2.2.1) Inverted index

Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word you want the list of documents in which it is contained in, and the relative *tfIdf* score.

__Tip__: *tfIdf* values are invariant with respect to the query, for this reason you can precalculate and store them accordingly.


In [14]:
import numpy as np

def create_inv_idx(corpus, vocab):
    tf = {}  
    #creating nested dictionary tf[word_idx][doc_num] = tf = freq of the word in this document/ quantity of all words in the doc 
    for word in vocab:
        tf[vocab[word]]={}     
    for doc_num, doc in enumerate(corpus):
        l = len(doc)
        for word in doc:
            if doc_num in tf[vocab[word]]:
                tf[vocab[word]][doc_num] += 1/l #dividing by l - to obtain tf score
            else:
                tf[vocab[word]][doc_num] = 1/l   
    idf = {i: np.log(len(corpus)/len(tf[i])) for i in tf}    #idf
    #from nested dictionary to dict of tuples and tfidf = tf*idf
    tfidf = {k: list((d, (t*(idf[k]))) for d,t in v.items()) for (k, v) in tf.items()}
    return tfidf, idf 

inv_idx, idf = create_inv_idx(df['synopsis_clean'], vocab)
save_dict_to_file(inv_idx, "inv_idx_tfldf.json")
save_dict_to_file(idf, "idf.json") #let's save idf too, as they are invariant and we will need them for query


### 2.2.2) Execute the query

In this new setting, given a query you get the right set of documents (i.e., those containing all the words in the query) and sort them according to their similairty to the query. For this purpose, as scoring function we will use the Cosine Similarity with respect to the *tfIdf* representations of the documents.

Given a query, that you let the user enter:
```
saiyan race
```
the search engine is supposed to return a list of documents, __ranked__ by their Cosine Similarity with respect to the query entered in input.

More precisely, the output must contain:
* `animeTitle`
* `animeDescription`
* `Url`
* The similarity score of the documents with respect to the query (float value between 0 and 1)


__Example Output__:

| animeTitle | animeDescription | Url | Similarity |
|:-------------------------------------:|:-----:|:---------------------------------------------------------------------:|------------|
| Fullmetal Alchemist: Brotherhood | ... | https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood | 0.96 |
| Gintama | ... | https://myanimelist.net/anime/28977/Gintama%C2%B0 | 0.92 |
| Shingeki no Kyojin Season 3 Part 2 | ... | https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2 | 0.87 |

In [15]:
import math

def tfidf_query(corpus, query, inv_idx, idf):
    """
    This functions receives query and outputs tfidfs for words in this query
    
    Arguments
        corpus
        query : list of words
        inv_idx 
        idf
    Returns 
        tfidf_q, tfidf_docs - tfidf for query and for documents,
        tfidf_q - array, ith elemnt tfidf for ith word in query in query
        tfidf_docs["doc_number"] - array, ith elemnt tfidf for ith ford in query in doc_number
        
    """
    #tfidf indexes of words in query for all documents, in format of nested dict
    tfidf_d = {q: dict((i, v) for (i, v) in inv_idx[q]) for q in query} 
    docs = set.intersection(*[set(i for (i, v) in inv_idx[q]) for q in query]) #list of docs that have all words from query
    idf = {q: idf[q] for q in query} #idf for words from query
    #let's count tfidf for query
    l = len(query)
    tf_q ={}
    for word in query:
        if word in tf_q:
            tf_q[word] += 1/l #dividing by l - to obtain tf score
        else:
            tf_q[word] = 1/l   
    tfidf_q = [tf_q[i]*idf[i] for i in tf_q]  #tfidf for query, array
    
    #creating tfidf for words in query for each doc that has all words from query
    
    tfidf_docs = {d : [0 for i in tf_q] for d in docs} #empty dict of arrays 
    for d in docs:
        for i, w in enumerate(tf_q):
            tfidf_docs[d][i] = tfidf_d[w][d]
            
    return tfidf_q, tfidf_docs

def cosine_similiarity(q, d):
    '''
    This function counts cosine similiarity between two vectors q and d
    '''
    return (sum([q[i]*d[i] for i in range(len(q))]) / math.sqrt(sum(qi * qi for qi in q) * sum(di * di for di in d)))

In [22]:
import heapq
k = 10 #let's k be 10
query = parse_query(input().split(), vocab)
if query:
    tfidf_q, tfidf_docs = tfidf_query(df['synopsis_clean'], query, inv_idx, idf)
    #definig heap to finding k docs with largest cos similiarity
    if len(tfidf_docs) < k:
        larg_doc = {d: cosine_similiarity(tfidf_q, tfidf_docs[d]) for d in tfidf_docs.keys()}
    else:
        heap = [(cosine_similiarity(tfidf_q, tfidf_docs[key]), key) for key,value in tfidf_docs.items()]
        larg_doc = {d: simil for (simil, d) in heapq.nlargest(k, heap)}
    df_entries = get_df_entries(df, set(larg_doc.keys()), simil = larg_doc)
    display(df_entries)

best movie


,animeTitle,animeDescription,animeUrl,Similarity
6384,Pokemon: Meloetta no Kirakira Recital,"The 24th Pikachu short, set to premiere alongi...",https://myanimelist.net/anime/15961/Shakotan★B...,1.000000
4632,Pokemon: Pikachu to Eievui Friends (Pokemon: E...,"The 25th Pikachu short, set to premiere alongs...",https://myanimelist.net/anime/12223/sCRYed_Alt...,1.000000
4093,Inazuma Eleven: Chou Jigen Dream Match,A new movie featuring a match between fan-vote...,https://myanimelist.net/anime/37210/Isekai_Mao...,1.000000
3549,Tenjou Tenge: The Past Chapter (TenjhoTenge: T...,Mitsuomi Takayanagi and Maya Natsume both want...,https://myanimelist.net/anime/41353/The_God_of...,1.000000
4327,Ichigo 100%: Koi ga Hajimaru?! Satsuei Gasshuk...,Continues from where Ichigo 100% TV leaves off...,https://myanimelist.net/anime/10794/IS__Infini...,1.000000
15084,Byulbyul Iyagi 2,This film consists of 6 animated shorts produc...,https://myanimelist.net/anime/32237/Burutabu-chan,1.000000
12612,Oshare Majo Love and Berry: Shiawase no Mahou,A movie based on the popular fashion card and ...,https://myanimelist.net/anime/35175/Osakini_Do...,0.945966
8356,Memories Off #5 Togireta Film The Animation,Kawai Haruto belongs to a cinema club to reali...,https://myanimelist.net/anime/36728/Mameneko,0.886179


## 3. DEFINE A NEW SCORE!

## 5. Algorithmic question
You consult for a personal trainer who has a back-to-back sequence of requests for appointments. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, [30, 50, 20] is an acceptable solution (of duration 100), but [30, 40, 50, 20] is not, because 30 and 40 are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is [40, 50, 20], of total duration 110.

* Write an algorithm that computes the acceptable solution with the longest possible duration.
* Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [ ]:
def algoritmo (array):
    a=0
    b=array[0]
    for elem in array[1:]:
        n=max(a+elem,b)
        a=b
        b=n
    return b

In [ ]:
array=[3,4,5,60,4]
algoritmo(array)

64